Collaborative Filtering v1 testing

Testiramo user-user collaborative filtering algoritam. Testiramo tako da za korisnika, odvojimo dio filmova koje je
pogledao(test split) i probamo pretpostaviti koju je ocjenu dao filmu. U user-item matrici, za svaki film iz
test splita, tražimo korisnike iz user-item matrice koji su pogledali film i koji su najsličniji našem korisniku
po cosine similarityu. Uzimamo prosjek ocjena od 4 najsličnija korisnika koji smo dobili ovim postupkom. Potom
računamo grešku pomoću RMSEa. Algoritam troši puno procerskog vremena i jako je spor pa smo računali grešku samo 
za prvih 20 korisnika.

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools

ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")


def testingForUser(n):
    # Makes dataframe, train test splits
    df = pd.DataFrame(columns=movies.loc[:, 'movieId'])
    userRating = ratings.query("userId == {}".format(n))
    user_train, user_test = train_test_split(userRating, test_size=0.3, random_state=42)
    mean_train = user_train['rating'].mean()

    # Defines user vector
    for ind, r in user_train.iterrows():
        df.loc[n, r['movieId']] = r['rating'] - mean_train

    # Defines user vector
    for index in range(600):
        if((index + 1) == n): continue
        otherRating = ratings.query("userId == {}".format(index + 1))
        mean = otherRating['rating'].mean()
        for ind, r in otherRating.iterrows():
            df.loc[index + 1, r['movieId']] = r['rating'] - mean
    data = df.fillna(0)

    user_train = np.array(data.loc[n, :])
    user_train = np.array([user_train])

    # Array for final user ratings for RMSE
    user_test_ratings = []
    # Array for final user ratings for RMSE
    other_user_ratings = []
    # Dictionary for defining movies that other users and our searching user watched
    dictionary_other_user_watched = {}
    for ind, r in user_test.iterrows():
        dictionary = {}
        # If user watched movie, calculates cosine similarity and puts in dictionary
        for index in range(600):
             if(data.loc[index + 1, r['movieId']] == 0): continue
             otherRating = np.array(data.loc[index + 1, :])
             otherRating = np.array([otherRating])

             result = cosine_similarity(user_train, otherRating)
             dictionary[index + 1] = result

        # Sorted dictionary to find user with best cosinesimilarity score
        sorted_dict = {}
        sorted_keys = sorted(dictionary, key=dictionary.get)
        sorted_keys = reversed(sorted_keys)
        for w in sorted_keys:
            sorted_dict[w] = dictionary[w]

        # Takes user with best cosine similarity score and takes his rating of movie
        x = itertools.islice(sorted_dict.items(), 0, 4)
        combine_score = 0
        for key, value in x:
            tmp = []
            tmp.append(ratings.query("userId == {} & movieId == {}".format(key, r['movieId'])))
            combine_score += tmp[0].values[0, 2]

        dictionary_other_user_watched[r['movieId']] = combine_score / 4


    #Fills test arrays with values
    for key, value in dictionary_other_user_watched.items():
        # index 0 user ID, index 1 movieId, index 3 rating
        other_user_ratings.append(value)
        user_test_ratings.append(user_test.query("movieId == {}".format(key))['rating'])

    return (mean_squared_error(user_test_ratings, other_user_ratings) ** 0.5)
    

average = 0
for i in range(20):
    tmp = testingForUser(i + 1)
    average += tmp
    print(tmp)
    
print(average / 20)
    

1.3442171115028787
1.1539485353438526
3.1932138330110833
1.4367263804593038
0.9437293044088437
1.2810614431215357
1.2943228882819136
1.1236102527122116
1.1436938901147082
1.7962634828999893
1.2051322541530451
0.9690523721657153
0.6543126164151201
0.8684276980075352
1.2355876453203523
0.7977833874764085
0.6778441238588707
1.0726410279466458
1.0767702827001502
1.159202311936963
1.2213770420918562


Zadnji redak pokazuje prosječnu pogrešku od 1.22.